In [1]:
import pandas as pd
import os

### Read Data 
Extract the data from excel file - the-office-lines.xlsx

In [2]:
directory = os.getcwd()
data = pd.read_excel(directory + '/the-office-lines.xlsx')
df =  pd.DataFrame(data, columns=['speaker', 'line_text'])
df.head()

,speaker,line_text
0,Michael,All right Jim. Your quarterlies look very good...
1,Jim,"Oh, I told you. I couldn't close it. So..."
2,Michael,So you've come to the master for guidance? Is ...
3,Jim,"Actually, you called me in here, but yeah."
4,Michael,"All right. Well, let me show you how it's done."


### Transcripts
Creating a transcript file for each character in the characters[ ]

In [5]:
import re
characters = ['Dwight', 'Pam']

for index, row in df.iterrows():
    if row['speaker'] in characters:
        speaker = row['speaker']
        with open('%s_transcript' %speaker + '.txt', 'a') as f:
            f.write(str(row['line_text']))
print('Done')

Done


### Actions
Extract actions from characters transcript files,
and saves them to at new actions file

In [8]:
mkdir pickle_transcript

In [9]:
import pickle

for character in characters:
    all_lines = open('%s_transcript' %character + '.txt', 'r')
    for line in all_lines:
        res = re.findall(r'\[.*?\]', line)
        if not len(res) == 0:
            with open('pickle_transcript/' +'%s_actions' %character + '.txt', 'wb') as f:
                pickle.dump(str(res), f)

### Lines
Extract Lines from characters transcript files,
and saves them to at new line file

In [10]:
for character in characters:
    all_lines = open('%s_transcript' %character + '.txt', 'r')
    for line in all_lines:
        res = re.sub(r'\[.*?\]', '', line)
        if not len(res) == 0:
            with open('pickle_transcript/' +'%s_lines' %character + '.txt', 'wb') as f:
                pickle.dump(str(res), f)

In [11]:
dataLines = {}
for character in characters:
    #print(character+'_actions'  + '.txt')
    with open('pickle_transcript/' + character + '_lines'  + '.txt', 'rb') as file:
        dataLines[character] = pickle.load(file)

In [20]:
#Combine the text with a space inbetween
def combine_text(list_of_text):
    combined_text = ''.join(list_of_text)
    return combined_text

In [21]:
data_combined = {key: [combine_text(value)] for (key, value) in dataLines.items()}
data_combined
print(dataLines)

{'Dwight': " Shall I play for you? Pa rum pump um pum  I have no gifts for you. Pa rum pump um pum Whassup!Whassup.What?Just clearing my desk. I can't concentrate.It's overlapping. It's all spilling over the edge. One word, two syllables. Demarcation.You can't do that.Safety violation. I could fall and pierce an organ.Downsizing?Downsizing? I have no problem with that. I have been recommending downsizing since I first got here. I even brought it up in my interview. I say, bring it on.I'm Assistant Regional Manager. I should know first.OK, um, can you just tell me please? Just tell me quietly. Can you whisper it in my ear?Please, OK? Do you want me to tell 'em?OK. You tell 'em. With my permission. Permission granted.Go ahead.On his mother's grave.Pam, information is power.Dwight Schrute, Assistant Regional Manager.Uh... yeah I got a '78 280Z. Bought it for $1,200. Fixed it up. It's now worth three grand.New engine, new suspension, I got a respray. I've got some photos.Damn it! Jim!He pu

In [22]:
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
Dwight,Shall I play for you? Pa rum pump um pum I have no gifts for you. Pa rum pump um pum Whassup!Whassup.What?Just clearing my desk. I can't concent...
Pam,"Well. I don't know.What?Uh, yeah. Just a fax.You haven't told me.Uh, yeah, the one...You put in the garbage can that was a special filing cabinet...."


In [23]:
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub("[']", '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    
    return text

round1 = lambda x: clean_text_round1(x)

In [24]:
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
Dwight,shall i play for you pa rum pump um pum i have no gifts for you pa rum pump um pum whassup whassup what just clearing my desk i cant concentr...
Pam,well i dont know what uh yeah just a fax you havent told me uh yeah the one you put in the garbage can that was a special filing cabinet i ...


In [25]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [26]:
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean
    
#Gem i fil

,transcript
Dwight,shall i play for you pa rum pump um pum i have no gifts for you pa rum pump um pum whassup whassup what just clearing my desk i cant concentr...
Pam,well i dont know what uh yeah just a fax you havent told me uh yeah the one you put in the garbage can that was a special filing cabinet i ...
